# NODE ANALYSIS PROGRAM

```
Python code to generate nodal equations from a circuit net list
by Tony
Date: April 17, 2017
Name: network.ipynb
Synopsis: This program will read in a spice type file and compute the node equations.

Description:

Requires: Python version 3 or higher
Author: Tony Cirineo
Revision History
7/1/2015: Ver 1 - coding started, derived from network.c code

Todo
need to merge DC and AC analysis, omega is the frequency variable

```

In [63]:
import os
import re as re
#import sympy as sympy
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
#init_printing()

#### open file and preprocess it
- remove blank lines and comments
- converts all lower case to upper
- removes extra spaces between entries
- count number of entries on each line, make sure the count is correct

In [64]:
fn = 'TEST2'
fd1 = open(fn+'.NET','r')
content = fd1.readlines()
content = [x.strip() for x in content]  #remove leading and trailing white space
# remove empty lines
while '' in content:
    content.pop(content.index(''))

# remove comment lines, these start with a asterisk *
content = [n for n in content if not n.startswith('*')]
# converts all lower case to upper
content = [x.upper() for x in content]
# removes extra spaces between entries
content = [' '.join(x.split()) for x in content]

In [65]:
branch_cnt = len(content)
# chech number of entries on each line
for i in range(branch_cnt):
    x = content[i][0]
    tk_cnt = len(content[i].split())

    if (x == 'R') or (x == 'L') or (x == 'C'):
        if tk_cnt != 4:
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("1had {:d} items and should only be 4".format(tk_cnt))
    elif (x == 'V') or (x == 'I'):
        if (tk_cnt != 6) and (tk_cnt != 7):
            print("branch {:d} not formatted correctly, {:s}".format(i,content[i]))
            print("2had {:d} items and should only be 6 or 7".format(tk_cnt))
    elif (x == 'E') or (x == 'F') or (x == 'G') or (x == 'H'):
        if (tk_cnt != 6):
            print("branch {:d} not formatted correctly, {}".format(i,content[i]))
            print("3had {:d} items and should only be 6".format(tk_cnt))
    else:
        print("unknown element type in branch {:d}, {}".format(i,content[i]))

#### PARSER
- puts branch elements into structure
- counts number of nodes

In [66]:
# try puting the branch structre in a pandas data frame
count = []
element = []   # type of element
p_node = []   # positive node
n_node = []  #  neg node
cp_node = []   # controlling pos node of branch
cn_node = [] # controlling neg node of branch
source_type = []  # 1 = AC, 2 = DC source, 3 for multi-terminal
value = []   # value of element or voltage
phase = []   # AC phase
source_imp = [] # source impedance

df = pd.DataFrame(index=count, columns=['element','p node','n node','cp node','cn node',
    'source type','value','phase','source imp'])

#### INDEPENDENT SOURCES
- functions to load AC and DC voltage and current sources into branch structure

In [67]:
# loads AC and DC voltage and current sources into branch structure
def indep_source(br_nu):
    tk = content[br_nu].split()
    df.loc[br_nu,'element'] = tk[0]
    df.loc[br_nu,'p node'] = int(tk[1])
    df.loc[br_nu,'n node'] = int(tk[2])
    df.loc[br_nu,'source type'] = tk[3]
    if tk[3] == 'AC':
        df.loc[br_nu,'value'] = float(tk[4])
        df.loc[br_nu,'phase'] = float(tk[5])
        df.loc[br_nu,'source imp'] = float(tk[6])
    if tk[3] == 'DC':
        df.loc[br_nu,'value'] = float(tk[4])
        df.loc[br_nu,'source imp'] = float(tk[6])

In [68]:
# loads passive elements into branch structure
def pass_element(br_nu):
    tk = content[br_nu].split()
    df.loc[br_nu,'element'] = tk[0]
    df.loc[br_nu,'p node'] = int(tk[1])
    df.loc[br_nu,'n node'] = int(tk[2])
    df.loc[br_nu,'value'] = float(tk[3])

In [69]:
'''
loads multi-terminal sub-networks
into branch structure
Types:
E - VCVS
G - VCCS
F - CCCS
H - CCVS
not implemented yet:
K - Coupled inductors
'''
def sub_network(br_nu):
    tk = content[br_nu].split()
    df.loc[br_nu,'element'] = tk[0]
    df.loc[br_nu,'p node'] = int(tk[1])
    df.loc[br_nu,'n node'] = int(tk[2])
    df.loc[br_nu,'cp node'] = int(tk[3])
    df.loc[br_nu,'cn node'] = int(tk[4])
    df.loc[br_nu,'value'] = float(tk[5])

In [70]:
# scan df and get largest node number
def count_nodes():
    # need to ckeck that nodes are consecutive
    # fill array with node numbers
    p = np.zeros(branch_cnt+1)
    for i in range(branch_cnt-1):
        p[df['p node'][i]] = df['p node'][i]
        p[df['n node'][i]] = df['n node'][i]

    # find the largest node number
    if df['n node'].max() > df['p node'].max():
        largest = df['n node'].max()
    else:
        largest =  df['p node'].max()

        largest = int(largest)
    # check for unfilled elements, skip node 0
    for i in range(1,largest):
        if p[i] == 0:
            print("nodes not in continuous order");

    return largest

In [71]:
# load branches into data frame
for i in range(branch_cnt):
    x = content[i][0]

    if (x == 'R') or (x == 'L') or (x == 'C'):
        pass_element(i)
    elif (x == 'V') or (x == 'I'):
        indep_source(i)
    elif (x == 'E') or (x == 'F') or (x == 'G') or (x == 'H'):
        sub_network(i)
    else:
        print("unknown element type in branch {:d}, {}".format(i,content[i]))

# count number of nodes
num_nodes = count_nodes()

In [72]:
num_nodes

10

In [49]:
df

,element,p node,n node,cp node,cn node,source type,value,phase,source imp
0,V1,1,0,NaN,NaN,AC,10,0,1
1,R1,10,3,NaN,NaN,NaN,1,NaN,NaN
2,R2,1,2,NaN,NaN,NaN,1,NaN,NaN
3,R3,2,5,NaN,NaN,NaN,1,NaN,NaN
4,R4,3,4,NaN,NaN,NaN,1,NaN,NaN
5,R5,5,7,NaN,NaN,NaN,1,NaN,NaN
6,R6,8,9,NaN,NaN,NaN,1,NaN,NaN
7,C1,1,10,NaN,NaN,NaN,1,NaN,NaN
8,C2,5,0,NaN,NaN,NaN,1,NaN,NaN
9,C3,4,6,NaN,NaN,NaN,1,NaN,NaN


In [73]:
# store the data frame as a pickle file
#build file name
df.to_pickle(fn+'.pkl')

#### Build the NAM
- load up network matrix and vectors
- load V & I vectors with vi_vector()
- load branch admittances with load_admit_list()
- build NAM and Y with build_nam()

top level calls are from dc_analysis() and dc_output(), need to generalized the AC and DC case

In [74]:
branch_cnt

16

In [75]:
# initialize voltage and current source vectors
# this function only does DC now, AC is a bit different, see C code
V = np.zeros(branch_cnt, dtype=complex)
I = np.zeros(branch_cnt, dtype=complex)

# calculates voltage and current vectors for node equations
def vi_vector():
    for i in range(branch_cnt):
        if df.loc[i,'element'] == 'V':
            if df.loc[i,'source type'] == 'DC':
                V[i] = df.loc[i,'value']
            else:
                V[i] = 0
        if df.loc[i,'element'] == 'I':
            if df.loc[i,'source type'] == 'DC':
                I[i] = -df.loc[i,'value']
            else:
                I[i] = 0

In [3]:
# initalize admmitance list, need to generalize for AC case
Y = np.zeros(branch_cnt, dtype=complex)
SHORT = 1E-10
OPEN = 1E+10
# formulates admittances list
def load_admit_list():
    #load R L & C branch values into Y and JY admittence lists
    for i in range(branch_cnt):
        x = df.loc[i,'element']
        if x == 'R':
            Y[i] = 1/df.loc[i,'value']
        elif x == 'L':
            Y[i] = OPEN   #for AC JY[i] = 1/(df.loc[i,'value']*omega*1J)
        elif x == 'C':
            Y[i] = SHORT  # df.loc[i,'value']*omega*1J;
        elif (x == 'V') or (x == 'I'): #put in source impedance
            Y[i] = 1/df.loc[i,'source imp']
            # take care of AC sources
            if df.loc[i,'type'] == 'AC':
                if x == 'V':  Y[i] = SHORT;
                if x == 'I':  Y[i] = OPEN;
            else:
                Y[i] = 1/df.loc[i,'source_imp']
        else:
            print('Problem loading R L & C\'s into admittance list')

NameError: name 'np' is not defined

In [90]:
V[0] = 1
Y[0]*V[0]

(1+1j)

In [1]:
'''
assembles NAM directly from circuit description
NAM - Re part of nodal admittance matrix
DV - Re part of the driving vector
Y - RE branch admittence list
UE - branch voltage source list
IE - branch current source list
'''
# initialize arrays
DV = np.zeros(branch_cnt, dtype=complex)
NAM = np.zeros((branch_cnt,branch_cnt), dtype=complex)

def build_nam():
    # build NAM and driving vector
    for k in range(branch_cnt):
        # skip past multi-terminal networks
        if df.loc[k,'source type'] == 3:  #need to check on source type
            continue

        p = df.loc[k,'p node']
        q = df.loc[k,'n node']
        # calculate NAM, can numpy do this math?
        if p > 0:
            NAM[p,p] = NAM[p,p] + Y[k]
        if q > 0:
            NAM[q,q] = NAM[q,q] + Y[k]
        if p > 0 and q > 0:
            NAM[p,q] = NAM[p,q] - Y[k]
            NAM[q,p] = NAM[q,p] - Y[k]

        # calculation of driving vector
        current = Y[k] * V[k] - I[k]
        if p > 0:
            DV[p] = DV[p] + current
        if q > 0:
            DV[q] = DV[q] - current

    # add sub-networks to NAM
    super_impos()

NameError: name 'np' is not defined

10000000000.0

In [56]:
'''
performs superposition of multi-terminal
elements into the admittence matrix

YM - multi-terminal matrix
lm = terminal/node association list

format of elements of multi-terminal matrixes
-------------------------------------------
| sign | g3 | g2 | g1 | p4 | p3 | p2 | p1 |
-------------------------------------------

sign - set to 1 if negative
g1-3 - additional parameters
p1-4 - gain factors
'''
def super_impos():
    print('not supported at this time')

'''
    /* if no sub-networks return */
    if(num_multi == 0)
        return;

    /* search thru network structure to find multi-terminal elements */
    for(k = 1,snc = 0;k <= num_branches;k++){
        if(branch[k-1].type  != 3)
            continue;

        /* build temporary multi-terminal matrixes */
        switch(branch[k-1].element[0]){
            case 'E': /* VCVS */
                ptr = mt_vcvs;
                break;
            case 'G': /* VCCS */
                ptr = mt_vccs;
                break;
            case 'F': /* CCCS */
                ptr = mt_cccs;
                break;
            case 'H': /* CCVS */
                ptr = mt_ccvs;
                break;
            default:
                printf("\r\nunknown element type in branch %d\r\n",k);
                puts("failed in super_impos");
                exit(7);
        }

        /* 1st element is size of square matrix */
        num_term = sub_net[snc].num_terminals = *ptr++;

        /* allocate space for node terminal association list and temp matrix */
        sub_net[snc].lm = ivector(1,num_term);
        sub_net[snc].mt = dmatrix(1,num_term,1,num_term);

        /* load multi-terminal matrix with network values */
        build_mt(sub_net[snc].mt,num_term,ptr,k);

        /* build node/terminal association list  */
        sub_net[snc].lm[1] = branch[k-1].cp_node;
        sub_net[snc].lm[2] = branch[k-1].cn_node;
        sub_net[snc].lm[3] = branch[k-1].n_node;
        sub_net[snc].lm[4] = branch[k-1].p_node;
        for(i = 5;i <= num_term;i++)
            sub_net[snc].lm[i] = -1;

        /* expand NAM and driving vector */
        for(i = 1;i <= num_term;i++){
            if(sub_net[snc].lm[i] < 0){
                /* isolated additional nodes */
                num_nodes += 1;     /* bump node count */
                expand_nam_dv();    /* grow NAM & DV */
                sub_net[snc].lm[i] = num_nodes;
                p = sub_net[snc].lm[i];
                /* zero added row and column */
                for(j = 1;j <= num_nodes;j++)
                    NAM[p][j] = 0.0;
                for(j = 1;j <= num_nodes;j++)
                    NAM[j][p] = 0.0;
                DV[p] = 0.0;
            }
        }

        /* super position of multi-terminal admittence matrix */
        for(i = 1;i <= num_term;i++){
            p = sub_net[snc].lm[i];
            if(p > 0){
                for(j = 1;j <= num_term;j++){
                    q = sub_net[snc].lm[j];
                    if(q > 0)
                        NAM[p][q] = NAM[p][q] + sub_net[snc].mt[i][j];
                }
            }
        }

        snc++;  /* bump index for sub-network struct */
    }
}
'''

'\n    /* if no sub-networks return */\n    if(num_multi == 0)\n        return;\n\n    /* search thru network structure to find multi-terminal elements */\n    for(k = 1,snc = 0;k <= num_branches;k++){\n        if(branch[k-1].type  != 3)\n            continue;\n\n        /* build temporary multi-terminal matrixes */\n        switch(branch[k-1].element[0]){\n            case \'E\': /* VCVS */\n                ptr = mt_vcvs;\n                break;\n            case \'G\': /* VCCS */\n                ptr = mt_vccs;\n                break;\n            case \'F\': /* CCCS */\n                ptr = mt_cccs;\n                break;\n            case \'H\': /* CCVS */\n                ptr = mt_ccvs;\n                break;\n            default:\n                printf("\r\nunknown element type in branch %d\r\n",k);\n                puts("failed in super_impos");\n                exit(7);\n        }\n\n        /* 1st element is size of square matrix */\n        num_term = sub_net[snc].num_ter

In [57]:
vi_vector()

In [58]:
load_admit_list()

Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list
Problem loading R L & C's into admittance list


In [59]:
build_nam()

not supported at this time


In [60]:
df.loc[1,'n node']

3

In [61]:
'''
DC ANALYSIS
SOLVED BY THE NODAL ADMITTANCE METHOD
Y - branch admittance list
I - independent current source
E - independent voltage source
NAM = nodal admittance matrix
DV - driving vector
'''

# load up network matrix and vectors
vi_vector()        # load V & I vectors
load_admit_list()  # load branch admittances
build_nam()        # build NAM and Y

    /* calculate branch voltages and currents */
    for(i = 1;i <= num_branches;i++){
        p = branch[i-1].p_node;
        q = branch[i-1].n_node;

        if(p > 0 && q == 0){
            BV[i] = NV[p] - V[i];
            BI[i] = Y[i] * BV[i];
        }

        if(q > 0 && p == 0){
            BV[i] =  - (NV[q] + V[i]);
            BI[i] = Y[i] * BV[i];
        }

        if(p > 0 && q > 0){
            BV[i] = NV[p] - NV[q] - V[i];
            BI[i] = Y[i] * BV[i];
        }
    }
    /* determine terminal currents of sub-networks */
    terminal_currents();

IndentationError: unexpected indent (<ipython-input-61-31d1005ca758>, line 16)